In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split, LeaveOneOut, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
df = pd.read_csv('IphoneData.csv')
newdf = df.drop(columns=['NAME', 'CO2E'])
X = newdf.to_numpy()
y = df['CO2E'].to_numpy()

In [11]:
from sklearn.pipeline import Pipeline


xgBoost_SearchSpace = {
    'booster': ['gblinear'],
    'n_estimators': [225],
    'learning_rate': [.225],
    'reg_lambda': [0, 0.05, .1],
    'reg_alpha': [0, 0.05, .1],
    'eval_metric': ['rmse', 'auc'],
    'feature_selector': ['cyclic', 'random', 'greedy', 'thrifty'],
    'objective': ['reg:squaredlogerror'],
    'updater': ['shotgun', 'coord_descent'],
    'top_k': [0, 1 , 2, 3]            
}

rf_SearchSpace = {
    'criterion': ['mse', 'friedman_mse', 'mae'],  # Splitting criterion
    'splitter': ['best', 'random'],               # Strategy to choose split at each node
    'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],      # Maximum depth of the tree
    'min_samples_split': [2, 5, 10, 15, 20],              # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4, 8, 16],                # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt', 'log2', None], # Number of features to consider for the best split
    'max_leaf_nodes': [None, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]        # Maximum number of leaf nodes in the tree
}

svr_SearchSpace = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1],
    'epsilon': [0.1, 0.2, 0.5]
}

lasso_SearchSpace = {
    'alpha': [.1, .125, .175, .2, .15, .18, .17, .165, .185],  # Different values for alpha
    'tol': [.0001],  # Different values for tolerance
    'max_iter': [2000],  # Different values for maximum iterations
    'selection': ['cyclic']
}


scoring = {
    'r2': 'r2',  
    'neg_mean_squared_error': 'neg_mean_squared_error'
}

searchSpace = {
    'polynomialfeatures__degree': [1, 2, 3, 4, 5],
    'linearregression__C': [0.1, 1, 10]
}

pipe = Pipeline([('poly_features', PolynomialFeatures()), ('linear_model', LinearRegression())])


In [12]:


GS = GridSearchCV(estimator=pipe,
                  param_grid= searchSpace,
                  scoring = 'neg_mean_absolute_error',
                  refit='neg_mean_absolute_error',
                  cv=5,
                  verbose=4)

In [13]:
GS.fit(X, y)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


ValueError: Invalid parameter 'linearregression' for estimator Pipeline(steps=[('poly_features', PolynomialFeatures()),
                ('linear_model', LinearRegression())]). Valid parameters are: ['memory', 'steps', 'verbose'].

In [ ]:
GS.best_params_

{'criterion': 'friedman_mse',
 'max_depth': 100,
 'max_features': 'sqrt',
 'max_leaf_nodes': 80,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'splitter': 'best'}

In [ ]:
GS.best_score_

-6.337777777777778

In [ ]:
GS.best_estimator_

XGBRegressor(base_score=None, booster='gblinear', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='auc', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.225, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=225, n_jobs=None,
             num_parallel_tree=None, objective='reg:squaredlogerror', ...)